# Machine learning with TensorFlow: multiple hidden layers

Date: August 21-23, 2018

Neural network: three hidden layers

Classification of human body motion:
- walking
- sitting down
- turning right while walking
- turning left while walking

In [92]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

## Preparing data set for training

In [93]:
SAMPLE_NUMS = 260  # per sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

TRAINING_SET = [['./20180821_walking_straight.csv', 0],
               ['./20180821_sitting.csv', 1],
               ['./20180821_turning_left.csv', 2],
               ['./20180821_turning_right.csv', 3],                
               ['./20180823_walking_straight.csv', 0],
               ['./20180823_sitting.csv', 1],
               ['./20180823_turning_left.csv', 2],
               ['./20180823_turning_right.csv', 3]]

TEST_SET = [['./20180822_walking_straight.csv', 0],
            ['./20180822_sitting.csv', 1],
            ['./20180822_turning_left.csv', 2],
            ['./20180822_turning_right.csv', 3]]

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

In [94]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df, lpf=False):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    if lpf:
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [95]:
training_set = []
test_set = []

def label(l):
    ll = [0,0,0,0]
    ll[l] = 1
    return ll

for file, l in TRAINING_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        training_set.append([df[df['id']==i], label(l)])
        
for file, l in TEST_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        test_set.append([df[df['id']==i], label(l)])
    
random.shuffle(training_set)
random.shuffle(test_set)

In [96]:
# use accel x axis and gyro z axis values for classification of human body motion
train_x, train_t = [], []
test_x, test_t = [], []
for df, label in training_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    train_x.append(values)
    train_t.append(label)
for df, label in test_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    test_x.append(values)
    test_t.append(label)

## TensorFlow

In [97]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(20180823)
tf.set_random_seed(20180823)

In [98]:
num_samples = SAMPLE_NUMS * 6
num_units1 = 256
num_units2 = 256
num_classes = 4

x = tf.placeholder(tf.float32, [None, num_samples])

w1 = tf.Variable(tf.truncated_normal([num_samples, num_units1]))
b1 = tf.Variable(tf.zeros([num_units1]))
y1 = tf.matmul(x, w1) + b1
hidden1 = tf.nn.relu(y1)

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
y2 = tf.matmul(hidden1, w2) + b2
hidden2 = tf.nn.tanh(y2)

w0 = tf.Variable(tf.zeros([num_units2, num_classes]))
b0 = tf.Variable(tf.zeros([num_classes]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

In [99]:
t = tf.placeholder(tf.float32, [None, num_classes])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [100]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [101]:
i = 0
for _ in range(2000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val, h1, h2, y1_, y2_ = sess.run([loss, accuracy, hidden1, hidden2, y1, y2], feed_dict={x:test_x, t:test_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))
#        for h in h1:
#            print([int(z*100) for z in h])
#        for y in y1_:
#            print([z for z in y])
#        for h in h2:
#            print([int(z*100) for z in h])

Step: 100, Loss: 11.323387145996094, Accuracy: 0.90625
Step: 200, Loss: 9.49685001373291, Accuracy: 0.9375
Step: 300, Loss: 8.7363862991333, Accuracy: 0.9375
Step: 400, Loss: 8.169761657714844, Accuracy: 0.9375
Step: 500, Loss: 7.905895709991455, Accuracy: 0.9375
Step: 600, Loss: 7.712455749511719, Accuracy: 0.9375
Step: 700, Loss: 7.554086685180664, Accuracy: 0.9375
Step: 800, Loss: 7.429012298583984, Accuracy: 0.9375
Step: 900, Loss: 7.327851295471191, Accuracy: 0.9375
Step: 1000, Loss: 7.244356155395508, Accuracy: 0.9375
Step: 1100, Loss: 7.174267292022705, Accuracy: 0.9375
Step: 1200, Loss: 7.1146111488342285, Accuracy: 0.9375
Step: 1300, Loss: 7.063236236572266, Accuracy: 0.9375
Step: 1400, Loss: 7.018558025360107, Accuracy: 0.9375
Step: 1500, Loss: 6.979382514953613, Accuracy: 0.9375
Step: 1600, Loss: 6.944777011871338, Accuracy: 0.9375
Step: 1700, Loss: 6.9140238761901855, Accuracy: 0.9375
Step: 1800, Loss: 6.886548042297363, Accuracy: 0.9375
Step: 1900, Loss: 6.861881256103516,

In [102]:
p_test = sess.run(p, feed_dict={x:test_x})
(p_test*100).astype(int)[:10]

array([[44, 53,  1,  0],
       [ 2, 96,  0,  0],
       [ 0,  0, 99,  0],
       [ 0,  0,  0, 99],
       [ 0,  0, 99,  0],
       [ 0,  0, 99,  0],
       [ 0,  0,  0, 99],
       [ 1, 75, 21,  2],
       [ 0,  0,  0, 99],
       [ 0,  0,  0, 99]])

In [103]:
test_t[:10]

[[0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1]]

In [104]:
sess.close()